# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import json
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv file from part 1
# Not sure if I should be using the cleaned one or the previous one, 
#but Im guessing the cleaned one
file = "../Output_Data/Clean_City_Data"
clean_df = pd.read_csv(file)
# clean_df has a 'unnamed' column, so remove it so it looks better
#clean_df.head()
weather_df = clean_df.drop(columns=["Unnamed: 0"])
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bluff,-46.60,168.33,50.0,69,93,14.00,NZ,1603135414
1,albany,42.60,-73.97,61.0,73,100,5.97,US,1603135620
2,astana,51.18,71.45,32.0,59,0,6.71,KZ,1603135494
3,mataura,-46.19,168.86,50.0,69,94,14.00,NZ,1603135690
4,hasaki,35.73,140.83,53.6,93,75,5.82,JP,1603135690


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Initiate variables
locations = weather_df[['Lat', 'Lng']].astype(float)
humidity = weather_df['Humidity'].astype(float)

In [4]:
# Add some specifications to heatmap
heatmap_specs = {
    'width': '1000px',
    'height': '500px',
    'margin': '0 auto 0 auto'
}

# Create map
fig = gmaps.figure(layout=heatmap_specs, zoom_level=2, center=(0,0))

# Add layer details
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=100, 
                                 point_radius=1)
fig.add_layer(heat_layer)

plt.savefig("../Images/humidty_heatmap.png")

fig

Figure(layout=FigureLayout(height='500px', margin='0 auto 0 auto', width='1000px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

# Set specifications
ideal_temp = (weather_df['Max Temp']>70) & (weather_df['Max Temp']<86)
ideal_wind = weather_df['Wind Speed']<10
ideal_humid = weather_df['Humidity']<50

# Collect all ideal specs
ideal_vaca = ideal_temp & ideal_wind & ideal_humid

# Create new df using collected specs
ideal_vaca_df = weather_df[ideal_vaca]
ideal_vaca_df = ideal_vaca_df.dropna()

# Limit the number of rows returned by your API requests to a reasonable number.
# I think this is what that means... by making sure only 7 rows are returned..
ideal_vaca_df.head(7)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,guerrero negro,27.98,-114.06,85.14,42,0,7.56,MX,1603135691
21,luau,-10.71,22.22,73.83,46,0,2.55,AO,1603135694
64,macaubas,-13.02,-42.70,82.09,48,94,4.05,BR,1603135702
78,khanpur,28.65,70.66,74.23,16,0,3.40,PK,1603135706
94,salalah,17.02,54.09,84.20,48,75,2.24,OM,1603135696
118,binga,-17.62,27.34,80.20,25,0,2.68,ZW,1603135714
158,teotitlan,18.13,-97.08,81.63,47,37,5.37,MX,1603135722


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df
# I think that means just change the name of the df..
hotel_df = ideal_vaca_df

# Add a "Hotel Name" column to the df
# Use empty quotes for initial value, since we don't have that data yet
hotel_df['Hotel Name'] = ""

hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,guerrero negro,27.98,-114.06,85.14,42,0,7.56,MX,1603135691,
21,luau,-10.71,22.22,73.83,46,0,2.55,AO,1603135694,
64,macaubas,-13.02,-42.70,82.09,48,94,4.05,BR,1603135702,
78,khanpur,28.65,70.66,74.23,16,0,3.40,PK,1603135706,
94,salalah,17.02,54.09,84.20,48,75,2.24,OM,1603135696,


In [7]:
# Hit the Google Places API for each city's coordinates
# Set parameters dictionary to search for hotels with 5000 meters
params = {
    "radius": 5000, 
    "types": "hotels", 
    "keyword": "hotel", 
    "key": g_key}

# Start a for loop using iterrows 
for index, row in hotel_df.iterrows():
    
    # First, get the lat and long coords from our df
    lat = row['Lat']
    lng = row['Lng']
    
    # Add a location parameter using lat and long that we just iterrated through
    params['location'] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params).json()
    
    # Store the first Hotel result into the DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except:
        print("Missing data")

Missing data
Missing data
Missing data
Missing data


In [8]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,guerrero negro,27.98,-114.06,85.14,42,0,7.56,MX,1603135691,Hotel Los Caracoles
21,luau,-10.71,22.22,73.83,46,0,2.55,AO,1603135694,Kimbo Hotel
64,macaubas,-13.02,-42.70,82.09,48,94,4.05,BR,1603135702,Vitória Palace Hotel
78,khanpur,28.65,70.66,74.23,16,0,3.40,PK,1603135706,Shakir Hotel & Bar BQ
94,salalah,17.02,54.09,84.20,48,75,2.24,OM,1603135696,Grand flora hotel salalah


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add some specifications to heatmap
# This part reminds me of CSS, syntax is basicly identical, which makes sense since we are styling an image
heatmap_specs = {
    'width': '1000px',
    'height': '500px',
    'margin': '0 auto 0 auto'
}

# Add marker layer ontop of heat map
fig = gmaps.figure(layout=heatmap_specs, zoom_level=2, center=(0,0))

hotel_markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(hotel_markers)

# Save figure
plt.savefig("../Images/hotel_heatmap.png")

# Display figure
fig

Figure(layout=FigureLayout(height='500px', margin='0 auto 0 auto', width='1000px'))

<Figure size 432x288 with 0 Axes>